<a href="https://colab.research.google.com/github/yinon2592/DL_Project_046211/blob/main/section_c_classifier_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
# drive.mount('/content/drive/my-drive/project_calculations')
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# !pip install transformers
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, GPT2Model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00


In [3]:
import torch
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
tokenizer.padding_side = "left" # Very Important
tokenizer.bos_token='<|startoftext|>'
tokenizer.eos_token='<|endoftext|>'
tokenizer.pad_token='<|pad|>'

configuration = GPT2Config.from_pretrained('gpt2-large', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained('gpt2-large', output_hidden_states=False)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))


section_a_all_loss, section_a_model_description = None, None
section_a_model_description

load_model_parameters = True
model_path = '/content/drive/My Drive/project_calculations/section_c_option_1_classifier/'
if os.path.exists(model_path + 'section_c_model.pth') and load_model_parameters:
  print("loading model parameters..")
  model.load_state_dict(torch.load(model_path + 'section_c_model.pth'))
  section_a_all_loss = pd.read_csv(model_path + 'section_c_all_loss.csv')
  section_a_model_description = pd.read_csv(model_path + 'section_c_model_description.txt')

In [5]:
class GPT2Dataset(Dataset):
    def __init__(self, txt_list, labels, tokenizer, gpt2_type="gpt2", max_length=768):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        self.labels = []

        for txt, sentiment in zip(txt_list, labels):
            sentiment_text = "positive" if sentiment == 1 else "negative"
            max_length = 150
            # truncated the txt
            txt = txt[:max_length]
            encodings_dict = self.tokenizer.encode_plus(f'\"{txt}\" Q: between positive or negative what was the sentiment of the last text ? A:',
                                                   text_pair=f'{sentiment}',
                                                   padding='max_length',
                                                   truncation=True,
                                                   max_length=max_length,
                                                   return_tensors='pt')

            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [6]:
test_data_path = '/content/drive/My Drive/project_dataset/test_data.csv'
df = pd.read_csv(test_data_path)
df = df.head(5000)
print("dataset size is ", df.shape[0])
print(df.label.value_counts())
print(df.sample(5), "\n")

dataset size is  5000
1    2534
0    2466
Name: label, dtype: int64
      label                                               text
4496      1  i ll try when i can next summon up the energy ...
3784      1  okie dokie just helped you out a bit more how ...
2615      0                                        whats wrong
315       1  at work protecting the innocent weak from the ...
2054      0  is wondering how a windows server acquired an ... 



In [7]:
# Tell pytorch to run this model on the GPU.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [8]:
model.eval()
def generate_response(question):
    # Encode the question
    input_ids = tokenizer.encode(question, return_tensors="pt").to(device)

    # Generate the response
    output = model.generate(
        input_ids,
        max_length=100,
        num_return_sequences=1,
        attention_mask=input_ids.ne(tokenizer.pad_token_id).to(device),
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and return the generated response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


# Example usage
txt = "i love you"
question = f'\"{txt}\" Q: between positive or negative what was the sentiment of the last text ? A:'
response = generate_response(question)
# response= response.split()[-1]
print(f"{question}\n{response}")

"i love you" Q: between positive or negative what was the sentiment of the last text ? A:
"i love you" Q: between positive or negative what was the sentiment of the last text? A: "i love you" Q: what was the last text? A: "i love you" Q: what was the last text? A: "i love you" Q: what was the last text? A: "i love you" Q: what was the last text? A: "i love you" Q: what was the last text? A: "i love


In [9]:
def test(df):
  generated_results = []
  for index, row in df.iterrows():
    # Access row values
    text_value = row['text']
    label_value = row['label']
    question = f'\"{txt}\" Q: between positive or negative what was the sentiment of the last text ? A:'
    input_ids = tokenizer.encode(question, return_tensors="pt").to(device)

    # Generate the response
    output = model.generate(
        input_ids,
        max_length=100,
        num_return_sequences=1,
        attention_mask=input_ids.ne(tokenizer.pad_token_id).to(device),
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and return the generated response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_results.append(response)
  return generated_results

generated_sentences = test(df)
result_csv = df.copy()
result_csv = result_csv[['text', 'label']]
result_csv['label'] = result_csv['label'].apply(lambda x: 'positive' if x == 1 else 'negative')

# Assuming you have already loaded your CSV file into a DataFrame called 'data'
result_csv ['prompt_mask'] = "{text} Q: between positive or negative what was the sentiment of the last text ? A:"
df['input_prompt'] = df['text'].apply(lambda text: f"{text} Q: between positive or negative what was the sentiment of the last text ? A:")
result_csv ['generated_sentence'] = generated_sentences
result_csv ['model'] = 'gpt2-large'


print(result_csv)
# Save the DataFrame to a CSV file
result_csv.to_csv('/content/drive/My Drive/project_dataset/section_c_option_1_prompt_1_classifier_results.csv', index=False)

KeyboardInterrupt: ignored